# Análise comparativa de modelos

In [3]:
import numpy as np
import pandas as pd

## 1. Obtenção dos dados

In [4]:
df = pd.read_csv("../data/raw/Sleep_health_and_lifestyle_dataset.csv")

In [5]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [8]:
dicionario = pd.read_csv("../data/external/dictionary.csv")
dicionario

,variavel,descrição,tipo,subtipo
0,Gender,O gênero da pessoa (Masculino/Feminino),qualitativa,nominal
1,Age,A idade da pessoa em anos,quantitativa,discreta
2,Occupation,A ocupação ou profissão da pessoa,qualitativa,nominal
3,Sleep Duration,O número de horas que a pessoa dorme por dia,quantitativa,contínua
4,Quality of Sleep,Uma classificação subjetiva da qualidade do so...,quantitativa,discreta
5,Physical Activity Level,O número de minutos que a pessoa pratica ativi...,quantitativa,discreta
6,Stress Level,Uma classificação subjetiva do nível de estres...,quantitativa,discreta
7,BMI Category,"A categoria de IMC da pessoa (por exemplo, Aba...",qualitativa,ordinal
8,Blood Pressure,"A medição da pressão arterial da pessoa, indic...",qualitativa,ordinal
9,Heart Rate,A frequência cardíaca em repouso da pessoa em ...,quantitativa,discreta


## 2. Preparação dados

### - Tratamento de dados faltantes

### - Tratamento de dados discrepantes